In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import nest_asyncio
nest_asyncio.apply()

from deepagents import create_deep_agent
from langchain_ollama import ChatOllama
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "mcp_zero_and_alita_rag": {
            "url": "http://localhost:8000/mcp",
            "transport": "streamable_http"
        }
    }
)
tools = await client.get_tools()

model = ChatOllama(
    model="gpt-oss:20b",
    temperature=0,
)

## Baseline

In [ ]:
from langgraph.prebuilt import create_react_agent

react_agent = create_react_agent(
    model=model,
    tools=tools
)

async for chunk in react_agent.astream(
    {
        "messages": [
            {"role": "user", "content": "How does Alita enable agents to create and deploy MCP tools if the agent deems that the tool does not exist?"}
        ]
    },
    stream_mode="values"
):
    if "messages" in chunk:
        chunk['messages'][-1].pretty_print()

================================ Human Message =================================

How does Alita enable agents to create and deploy MCP tools if the agent deems that the tool does not exist?
================================== Ai Message ==================================
Tool Calls:
  alita_documentation (8fa8565d-6f78-4eb2-8f63-10229df3ab17)
 Call ID: 8fa8565d-6f78-4eb2-8f63-10229df3ab17
  Args:
    query: How does Alita enable agents to create and deploy MCP tools if the agent deems that the tool does not exist?
================================= Tool Message =================================
Name: alita_documentation

<think>
Okay, let's tackle this query. The user is asking how Alita enables agents to create and deploy MCP tools when the tool doesn't exist. I need to go through the context provided and extract the relevant information.

First, I remember from the context that Alita uses MCP Brainstorming, ScriptGeneratingTool, and CodeRunningTool. The MCP Brainstorming is part of th

## Running our first deep agent without sub agents

In [4]:
base_deep_agent = create_deep_agent(
    model=model,
    tools=tools,
    instructions="""
    You are an expert in the domain of MCP Zero and Alita. Your job is write a comprehensive report through
    thorough research and analysis. 
    
    Here are the instructions for writing the final report:

    <report_instructions>

    CRITICAL: Make sure the answer is written in the same language as the human messages! If you make a todo plan - you should note in the plan what language the report should be in so you dont forget!
    Note: the language the report should be in is the language the QUESTION is in, not the language/country that the question is ABOUT.

    Please create a detailed answer to the overall research brief that:
    1. Is well-organized with proper headings (# for title, ## for sections, ### for subsections)
    2. Includes specific facts and insights from the research
    3. Provides a balanced, thorough analysis. Be as comprehensive as possible, and include all information that is relevant to the overall research question. People are using you for deep research and will expect detailed, comprehensive answers.
    You can structure your report in a number of different ways. Here are some examples:

    To answer a question that asks you to compare two things, you might structure your report like this:
    1/ intro
    2/ overview of topic A
    3/ overview of topic B
    4/ comparison between A and B
    5/ conclusion

    To answer a question that asks you to return a list of things, you might only need a single section which is the entire list.
    1/ list of things or table of things
    Or, you could choose to make each item in the list a separate section in the report. When asked for lists, you don't need an introduction or conclusion.
    1/ item 1
    2/ item 2
    3/ item 3

    To answer a question that asks you to summarize a topic, give a report, or give an overview, you might structure your report like this:
    1/ overview of topic
    2/ concept 1
    3/ concept 2
    4/ concept 3
    5/ conclusion

    If you think you can answer the question with a single section, you can do that too!
    1/ answer

    REMEMBER: Section is a VERY fluid and loose concept. You can structure your report however you think is best, including in ways that are not listed above!
    Make sure that your sections are cohesive, and make sense for the reader.

    For each section of the report, do the following:
    - Use simple, clear language
    - Use ## for section title (Markdown format) for each section of the report
    - Do NOT ever refer to yourself as the writer of the report. This should be a professional report without any self-referential language. 
    - Do not say what you are doing in the report. Just write the report without any commentary from yourself.
    - Each section should be as long as necessary to deeply answer the question with the information you have gathered. It is expected that sections will be fairly long and verbose. You are writing a deep research report, and users will expect a thorough answer.
    - Use bullet points to list out information when appropriate, but by default, write in paragraph form.

    REMEMBER:
    The brief and research may be in English, but you need to translate this information to the right language when writing the final answer.
    Make sure the final answer report is in the SAME language as the human messages in the message history.
    </report_instructions>

    You have access to a few tools. Namely:
    ## `alita_documentation`
    ## `mcp_zero_documentation`

    Use these tools to get information about Alita and MCP Zero. 
    """
)

In [ ]:
async for chunk in base_deep_agent.astream(
    {
        "messages": [
            {"role": "user", "content": "How does Alita enable agents to create and deploy MCP tools if the agent deems that the tool does not exist?"}
        ]
    },
    stream_mode="values"
):
    if "messages" in chunk:
        chunk['messages'][-1].pretty_print()

## Running with sub agents

In [11]:
alita_mcp_zero_prompt = """
You are an expert in all matters relating to MCP Zero and Alita. Use the tools you have 
available to answer the user's question. Do not answer the questions directly. Always use the tools.

Write your answer in a report format with sections and subsections. Be crisp in your writing.
"""

alita_mcp_zero_sub_agent = {
    "name": "alita-mcp-zero-agent",
    "description": "Used to answer questions about MCP Zero and Alita.",
    "prompt": alita_mcp_zero_prompt,
    "tools": tools,
    "model_settings": {
        "model": "ollama:qwen3:latest",
        "temperature":0,
    } 
}

In [12]:
critiquer_prompt = """
You are a dedicated editor tasked to critique a report. You can find the report at 
`final_report.md`. You can find the question/topic for this report at `question.txt`.
The user may ask for specific areas to critique the report in. Respond to the user with
a detailed critique of the report and suggest improvements. Be constructive in your critique.

Do not write the `final_report.md` file yourself.

Things to check:
- Check that each section is appropriately named
- Check that the report is written as you would find in an essay or a textbook - it should be text heavy, do not let it just be a list of bullet points!
- Check that the report is comprehensive. If any paragraphs or sections are short, or missing important details, point it out.
- Check that the article covers key areas of the industry, ensures overall understanding, and does not omit important parts.
- Check that the article deeply analyzes causes, impacts, and trends, providing valuable insights
- Check that the article closely follows the research topic and directly answers questions
- Check that the article has a clear structure, fluent language, and is easy to understand.
"""

critique_sub_agent = {
    "name": "critiquer-agent",
    "description": "Used to critique the final report. Give this agent some information about how you want it to critique the report.",
    "prompt": critiquer_prompt,
    "model_settings": {
        "model": "ollama:qwen3:latest",
        "temperature":0,
    } 
}

In [13]:
genai_instructions = """
You are an expert in the domain of MCP Zero and Alita. Your job is write a comprehensive report through
thorough research and analysis.

The first thing you should do is to write the original user's question to `question.txt` to keep
a record.

Then use the alita-mcp-zero-agent to conduct deep research on topics relating to Alita and MCP Zero.

When you think you have enough information to write a final report, write it to `final_report.md`.

You can call the critiquer-agent to critique the report and suggest improvements. After that (if needed),
you can do another round of research with the alita-mcp-zero-agent to improve the report. You can 
do this however many times you want until you are satisfied with the report.

Only edit the file once at a time (if you call this tool in parallel, there may be conflicts).

Here are the instructions for writing the final report:

<report_instructions>

CRITICAL: Make sure the answer is written in the same language as the human messages! If you make a todo plan - you should note in the plan what language the report should be in so you dont forget!
Note: the language the report should be in is the language the QUESTION is in, not the language/country that the question is ABOUT.

Please create a detailed answer to the overall research brief that:
1. Is well-organized with proper headings (# for title, ## for sections, ### for subsections)
2. Includes specific facts and insights from the research
3. Provides a balanced, thorough analysis. Be as comprehensive as possible, and include all information that is relevant to the overall research question. People are using you for deep research and will expect detailed, comprehensive answers.
You can structure your report in a number of different ways. Here are some examples:

To answer a question that asks you to compare two things, you might structure your report like this:
1/ intro
2/ overview of topic A
3/ overview of topic B
4/ comparison between A and B
5/ conclusion

To answer a question that asks you to return a list of things, you might only need a single section which is the entire list.
1/ list of things or table of things
Or, you could choose to make each item in the list a separate section in the report. When asked for lists, you don't need an introduction or conclusion.
1/ item 1
2/ item 2
3/ item 3

To answer a question that asks you to summarize a topic, give a report, or give an overview, you might structure your report like this:
1/ overview of topic
2/ concept 1
3/ concept 2
4/ concept 3
5/ conclusion

If you think you can answer the question with a single section, you can do that too!
1/ answer

REMEMBER: Section is a VERY fluid and loose concept. You can structure your report however you think is best, including in ways that are not listed above!
Make sure that your sections are cohesive, and make sense for the reader.

For each section of the report, do the following:
- Use simple, clear language
- Use ## for section title (Markdown format) for each section of the report
- Do NOT ever refer to yourself as the writer of the report. This should be a professional report without any self-referential language. 
- Do not say what you are doing in the report. Just write the report without any commentary from yourself.
- Each section should be as long as necessary to deeply answer the question with the information you have gathered. It is expected that sections will be fairly long and verbose. You are writing a deep research report, and users will expect a thorough answer.
- Use bullet points to list out information when appropriate, but by default, write in paragraph form.

REMEMBER:
The brief and research may be in English, but you need to translate this information to the right language when writing the final answer.
Make sure the final answer report is in the SAME language as the human messages in the message history.
</report_instructions>

You have access to a few tools. Namely:
## `alita_documentation`
## `mcp_zero_documentation`

Use these tools to get information about Alita and MCP Zero. 
"""

In [18]:
base_deep_agent = create_deep_agent(
    model=model,
    tools=tools,
    instructions="""
    You are an expert in the domain of MCP Zero and Alita. Your job is write a comprehensive report through
    thorough research and analysis. 
    
    Here are the instructions for writing the final report:

    <report_instructions>

    CRITICAL: Make sure the answer is written in the same language as the human messages! If you make a todo plan - you should note in the plan what language the report should be in so you dont forget!
    Note: the language the report should be in is the language the QUESTION is in, not the language/country that the question is ABOUT.

    Please create a detailed answer to the overall research brief that:
    1. Is well-organized with proper headings (# for title, ## for sections, ### for subsections)
    2. Includes specific facts and insights from the research
    3. Provides a balanced, thorough analysis. Be as comprehensive as possible, and include all information that is relevant to the overall research question. People are using you for deep research and will expect detailed, comprehensive answers.
    You can structure your report in a number of different ways. Here are some examples:

    To answer a question that asks you to compare two things, you might structure your report like this:
    1/ intro
    2/ overview of topic A
    3/ overview of topic B
    4/ comparison between A and B
    5/ conclusion

    To answer a question that asks you to return a list of things, you might only need a single section which is the entire list.
    1/ list of things or table of things
    Or, you could choose to make each item in the list a separate section in the report. When asked for lists, you don't need an introduction or conclusion.
    1/ item 1
    2/ item 2
    3/ item 3

    To answer a question that asks you to summarize a topic, give a report, or give an overview, you might structure your report like this:
    1/ overview of topic
    2/ concept 1
    3/ concept 2
    4/ concept 3
    5/ conclusion

    If you think you can answer the question with a single section, you can do that too!
    1/ answer

    REMEMBER: Section is a VERY fluid and loose concept. You can structure your report however you think is best, including in ways that are not listed above!
    Make sure that your sections are cohesive, and make sense for the reader.

    For each section of the report, do the following:
    - Use simple, clear language
    - Use ## for section title (Markdown format) for each section of the report
    - Do NOT ever refer to yourself as the writer of the report. This should be a professional report without any self-referential language. 
    - Do not say what you are doing in the report. Just write the report without any commentary from yourself.
    - Each section should be as long as necessary to deeply answer the question with the information you have gathered. It is expected that sections will be fairly long and verbose. You are writing a deep research report, and users will expect a thorough answer.
    - Use bullet points to list out information when appropriate, but by default, write in paragraph form.

    REMEMBER:
    The brief and research may be in English, but you need to translate this information to the right language when writing the final answer.
    Make sure the final answer report is in the SAME language as the human messages in the message history.
    </report_instructions>

    You have access to a few tools. Namely:
    ## `alita_documentation`
    ## `mcp_zero_documentation`

    Use these tools to get information about Alita and MCP Zero. 
    """
)

================================ Human Message =================================

How does Alita enable agents to create and deploy MCP tools if the agent deems that the tool does not exist?
================================== Ai Message ==================================
Tool Calls:
  alita_documentation (c8bdd057-c2c4-46e9-a67e-9f7dc62391b0)
 Call ID: c8bdd057-c2c4-46e9-a67e-9f7dc62391b0
  Args:
    query: Alita dynamic tool creation MCP tools agent creates tool if not exist
================================= Tool Message =================================
Name: alita_documentation

Error: ToolException("Error calling tool 'alita_documentation': Unknown index name alita-paper. Please confirm an index with this name exists.")
 Please fix your mistakes.
================================== Ai Message ==================================
Tool Calls:
  alita_documentation (8effabdf-6727-4e4b-8825-01b3ab2ef862)
 Call ID: 8effabdf-6727-4e4b-8825-01b3ab2ef862
  Args:
    query: Alita dynamic tool 

CancelledError: 

In [15]:
agent = create_deep_agent(
    model = model,
    tools = tools,
    instructions = genai_instructions,
    subagents = [alita_mcp_zero_sub_agent, critique_sub_agent],
).with_config({"recursion_limit": 1000})

TypeError: unhashable type: 'StructuredTool'